С помощью инстумента Constraint Optimization придумать условия и решить задачу расчета расписания преподавателей в школе или вузе. Есть 3 предмета, по каждому по 5 занятий в неделю у каждой из трех групп студентов. Одно занятие утром, одно днем и одно вечером. Часть преподавателей могут по четным числам, часть по утрам, часть только максимально 2 урока в неделю, часть ведут только 1 или 2 предмета из 3х (предложить от себя список таких преподавателей с ограничениями). Нужно найти решение с минимальным количеством задействованных преподавателей (с помощью инструмента Google OR tools.

То есть:
```
3 предмета (Math, Rus, Eng)
3 группы (A, B, C)
5 занятий 
Утро  - 1 занятие
День  - 1 занятие
Вечер - 1 занятие
```

Мои преподователи:
```
Фарпат - ведет только по четным числам
Гузперт - ведет только по вечерам
Йошки - ведет только математику и английский (Math, Eng)
Амред - ведет только 2 урока в неделю
Артем - если приходит работать, то только на утро и вечер, чтобы днем можно было выпить чашечку кофе
Андрей - работает не более 6 раз в неделю и ведет только русский (Rus)
Егор - работает только утром ```


In [1]:
from ortools.sat.python import cp_model

In [2]:
all_teachers = ["Фарпат", "Гузперт", "Йошки", "Амред", "Артем", "Андрей", "Егор"]
all_subjects = ["math", "rus", "eng"]
all_day_parts = ["утро", "день", "вечер"]
all_groups = ["a", "b", "c"]

num_teachers = len(all_teachers)
num_subjects = len(all_subjects)
num_day_parts = len(all_day_parts)
num_groups = len(all_groups)

num_days = 6
num_classes_each = 5

all_days = range(num_days)

In [3]:
model = cp_model.CpModel()

In [4]:
# Дни хранятся в переменных как индексы, но в дебаге показываются как обычные числа

classes = {}
for t in all_teachers:
    for d in all_days:
        for p in all_day_parts:
            for g in all_groups:
                for s in all_subjects:            
                    classes[(t, d, p, 
                            g, s)] =  model.NewBoolVar(f"Учитель <{t}>, день <{d+1}>, часть дня <{p}>, группа <{g}>, предмет <{s}>")

### Каждый урок должен вестись максимально 1 учителем

In [5]:
for d in all_days:
    for p in all_day_parts:
        for g in all_groups:
            for s in all_subjects:
                model.AddAtMostOne(classes[(t, d, p, g, s)] for t in all_teachers) 

### Учитель не может вести одновременно у нескольких групп или несколько предметов

In [6]:
for t in all_teachers:
    for d in all_days:
        for p in all_day_parts:
            model.Add(sum(classes[(t, d, p, g, s)] for s in all_subjects for g in all_groups) <= 1)            

### Одна группа не может быть на нескольких предметах 

In [7]:
for g in all_groups:
    for p in all_day_parts:
        for d in all_days:
            model.Add(sum(classes[(t, d, p, g, s)] for s in all_subjects for t in all_teachers) <= 1)  

###  Здесь проверяем, что это хоть как-то реально с нашим кол-вом учителей
###### Сумма Предметов * Сумма Групп * Сумма Занятий На Предмет

In [8]:
classes_num = num_subjects * num_classes_each * num_groups 
print(classes_num)
# Если бы они работали в любое время
max_classes_possible_by_teachers = num_teachers * num_day_parts * num_days
print(max_classes_possible_by_teachers)

45
126


### Сумма уроков по каждому предмету в каждой группе должна равняться 5 = num_classes_each

In [9]:
for s in all_subjects:
    for g in all_groups:
        model.Add(
            sum(classes[(t, d, p, g, s)]
            for t in all_teachers for d in all_days for p in all_day_parts
        ) == num_classes_each) 
#  Последняя строка обозначает, что совокупно со всеми учителями, всеми днями и частями дня, по одной группе и 
#  одному предмету должно быть 5 занятий

### Фарпат - ведет только по четным числам 

In [10]:
classes_Farpat_odd_days = []

for d in all_days:
    for p in all_day_parts:
        for g in all_groups:
            for s in all_subjects:
                if (d + 1) % 2 == 1: # Если день нечетный
                    classes_Farpat_odd_days.append(classes[("Фарпат", d, p, g, s)])
model.Add(sum(classes_Farpat_odd_days) == 0)

### Гузперт - ведет только по вечерам

In [11]:
classes_Guzpert_not_evening = []

for d in all_days:
    for p in all_day_parts:
        for g in all_groups:
            for s in all_subjects:
                if p != "вечер": 
                    classes_Guzpert_not_evening.append(classes[("Гузперт", d, p, g, s)])
model.Add(sum(classes_Guzpert_not_evening) == 0)

### Йошки - ведет только математику и английский (Math, Eng) 

In [12]:
model.Add(sum(classes[(
    "Йошки", d, p, g, "rus")] for d in all_days for p in all_day_parts for g in all_groups) == 0)

### Амред - ведет только 2 урока в неделю

In [13]:
classes_Amred = []

for d in all_days:
    for p in all_day_parts:
        for g in all_groups:
            for s in all_subjects:
                classes_Amred.append(classes[("Амред", d, p, g, s)])
model.Add(sum(classes_Amred) <= 2)

### Артем - если приходит работать, то только на утро и вечер, чтобы днем можно было выпить чашечку кофе 


In [14]:
classes_Artem_afternoon = []

for d in all_days:
    for g in all_groups:
        for s in all_subjects:
            b = model.NewBoolVar("")
            model.Add(classes[("Артем", d, "утро", g, s)] + classes[("Артем", d, "вечер", g, s)] == 0).OnlyEnforceIf(b)
            model.Add(classes[("Артем", d, "утро", g, s)] + classes[("Артем", d, "вечер", g, s)] == 2).OnlyEnforceIf(b.Not())

model.Add(sum(classes[("Артем", d, "день", g, s)] for d in all_days for g in all_groups for s in all_subjects) == 0)

### Андрей - работает не более 6 раз в неделю и ведет только русский (Rus)

In [15]:
classes_Andrey = []
classes_Andrey_not_rus = []

for d in all_days:
    for p in all_day_parts:
        for g in all_groups:
            classes_Andrey_not_rus.extend((classes[("Андрей", d, p, g, "math")], classes[("Андрей", d, p, g, "eng")]))
            for s in all_subjects:
                classes_Andrey.append(classes[("Андрей", d, p, g, s)])
                
model.Add(sum(classes_Andrey_not_rus) == 0)
model.Add(sum(classes_Andrey) <= 6)

### Егор - работает только утром

In [16]:
classes_Egor_not_morn = []

for d in all_days:
    for p in all_day_parts:
        for g in all_groups:
            for s in all_subjects:
                if p != "утро": 
                    classes_Egor_not_morn.append(classes[("Егор", d, p, g, s)])
model.Add(sum(classes_Egor_not_morn) == 0)

In [17]:
 class TeachersPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, classes, teachers, num_days, day_parts, subjects, groups, limit, objective):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._classes = classes
            self._groups = groups
            self._subjects = subjects
            self._teachers = teachers
            self._num_days = num_days
            self._day_parts = day_parts
            self._solution_count = 0
            self._solution_limit = limit
            self._objective = objective

        def on_solution_callback(self):
            print(f"\nКоличество задействованных учителей: {num_teachers - self.Value(self._objective)}")
            
            self._solution_count += 1
            print(f'Solution {self._solution_count}')
            for d in range(self._num_days):
                print(f'Day {d+1}')
                for t in self._teachers:
                    is_working = False
                    for s in self._subjects:
                        for g in self._groups:
                            for p in self._day_parts:
                                if self.Value(self._classes[(t, d, p, g, s)]):
                                    is_working = True
                                    print(f"Учитель <{t}>, день <{d+1}>, часть дня <{p}>, группа <{g}>, предмет <{s}>")
                    if not is_working:
                        print(f"Учитель {t} не работает")
            if self._solution_count >= self._solution_limit:
                print(f'Stop search after {self._solution_limit} solutions')
                self.StopSearch()

        def solution_count(self):
            return self._solution_count

In [18]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True
# Force the solver to follow the decision strategy not strictly.
solver.parameters.search_branching = cp_model.AUTOMATIC_SEARCH


### Нужно минимизировать количество работающих учителей. (Или максимизировать учителей, которые не работают)

In [19]:
# Вот тут важно. Создаем словарь с классами по каждому учителю

classes_by_teacher = {}
for t in all_teachers:
    cur_classes = []
    for d in all_days:
        for p in all_day_parts:
            for g in all_groups:
                for s in all_subjects:
                    cur_classes.append(classes[(t, d, p, g, s)])
    classes_by_teacher[t] = cur_classes
    
# Затем устанавливаем objective - это наша цель, которую мы будем максимизировать (не работающие учителя)
                    
objective = model.NewIntVar(0, num_teachers, "objective")

# мы собираем массив, с булевыми переменными, где каждое обозначает, не работает ли учитель под индексом i или работает
teacher_i_doesnt_work = []
for t in all_teachers:
    b = model.NewBoolVar(f'bool для учителя {t}')
    
    teacher_works_n_times = model.NewIntVar(0,10000, f"Сколько раз работает учитель {t}")
    model.Add(teacher_works_n_times == sum(classes_by_teacher[t]))
    
    teacher_doesnt_work = model.NewBoolVar(f"Учитель {t} не работает")
    
    # b == (teacher_works_n_times == 0).
    model.Add(teacher_works_n_times == 0).OnlyEnforceIf(b)
    model.Add(teacher_works_n_times != 0).OnlyEnforceIf(b.Not())

    # Во-первых b -> teacher_doesnt_work == 1.
    model.Add(teacher_doesnt_work  == True).OnlyEnforceIf(b)
    # Во-вторых !b -> teacher_doesnt_work == 0
    model.Add(teacher_doesnt_work == False).OnlyEnforceIf(b.Not())
    
    teacher_i_doesnt_work.append(teacher_doesnt_work)

# привязываем цель к сумме учителей, которые не работают и просим модель максимизировать их число
model.Add(objective == sum(teacher_i_doesnt_work))

model.Maximize(objective)

In [20]:
# Display the first <solution_limit> solutions.
solution_limit = 5
solution_printer = TeachersPartialSolutionPrinter(classes, all_teachers, num_days, all_day_parts,
                                                  all_subjects, all_groups, solution_limit, objective)

```
Фарпат - ведет только по четным числам
Гузперт - ведет только по вечерам
Йошки - ведет только математику и английский (Math, Eng)
Амред - ведет только 2 урока в неделю
Артем - если приходит работать, то только на утро и вечер, чтобы днем можно было выпить чашечку кофе
Андрей - работает не более 6 раз в неделю и ведет только русский (Rus)
Егор - работает только утром
```

### Как можно заметить, пролистав решения вниз до "Solution 4", CP-SAT solver не находит никаких решений лучше, чем 4 работающих учителя с данным набором учителей. То есть алгоритм довольно долго может быть запущен, но никаких вариантов он не выдает, и если его приостановить, статус будет Infeasible.

In [ ]:
status = solver.Solve(model, solution_printer)

if status == cp_model.OPTIMAL:
    print("\nOptimal")
elif status == cp_model.FEASIBLE:
    print("\nFeasible")
else:
    print("\nInfeasible")


Количество задействованных учителей: 7
Solution 1
Day 1
Учитель Фарпат не работает
Учитель <Гузперт>, день <1>, часть дня <вечер>, группа <a>, предмет <rus>
Учитель <Йошки>, день <1>, часть дня <день>, группа <b>, предмет <math>
Учитель <Амред>, день <1>, часть дня <вечер>, группа <b>, предмет <eng>
Учитель Артем не работает
Учитель <Андрей>, день <1>, часть дня <утро>, группа <a>, предмет <rus>
Учитель <Андрей>, день <1>, часть дня <день>, группа <c>, предмет <rus>
Учитель <Егор>, день <1>, часть дня <утро>, группа <c>, предмет <math>
Day 2
Учитель <Фарпат>, день <2>, часть дня <день>, группа <b>, предмет <rus>
Учитель <Фарпат>, день <2>, часть дня <вечер>, группа <b>, предмет <rus>
Учитель <Фарпат>, день <2>, часть дня <утро>, группа <b>, предмет <eng>
Учитель <Гузперт>, день <2>, часть дня <вечер>, группа <c>, предмет <rus>
Учитель Йошки не работает
Учитель Амред не работает
Учитель <Артем>, день <2>, часть дня <утро>, группа <a>, предмет <eng>
Учитель <Артем>, день <2>, часть дня 


Количество задействованных учителей: 4
Solution 4
Day 1
Учитель Фарпат не работает
Учитель Гузперт не работает
Учитель <Йошки>, день <1>, часть дня <вечер>, группа <a>, предмет <math>
Учитель <Йошки>, день <1>, часть дня <день>, группа <b>, предмет <eng>
Учитель <Йошки>, день <1>, часть дня <утро>, группа <c>, предмет <eng>
Учитель Амред не работает
Учитель <Артем>, день <1>, часть дня <утро>, группа <b>, предмет <math>
Учитель <Артем>, день <1>, часть дня <вечер>, группа <b>, предмет <math>
Учитель <Андрей>, день <1>, часть дня <утро>, группа <a>, предмет <rus>
Учитель <Андрей>, день <1>, часть дня <день>, группа <c>, предмет <rus>
Учитель Егор не работает
Day 2
Учитель <Фарпат>, день <2>, часть дня <день>, группа <a>, предмет <rus>
Учитель <Фарпат>, день <2>, часть дня <вечер>, группа <b>, предмет <rus>
Учитель <Фарпат>, день <2>, часть дня <утро>, группа <b>, предмет <eng>
Учитель Гузперт не работает
Учитель <Йошки>, день <2>, часть дня <утро>, группа <c>, предмет <eng>
Учитель <Йо